In [34]:
import numpy as np
import time
#%tensorflow_version 1.14
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
import os, time  
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from matplotlib import pyplot as plt
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#!unzip -uq "/content/drive/My Drive/img_align_celeba.zip" -d "/content/drive/My Drive/CelebA"

#dir_data      = "C:/Users/DalilR/Desktop/Prjt/img_align_celeba"
Ntrain        = 200000 
Ntest         = 100
#nm_imgs       = np.sort(os.listdir(dir_data))
## name of the jpg files for training set
#nm_imgs_train = nm_imgs[:Ntrain]
## name of the jpg files for the testing data
#nm_imgs_test  = nm_imgs[Ntrain:Ntrain + Ntest]
img_shape     = (64, 64, 3)

def get_npdata(nm_imgs_train):
    X_train = []
    for i, myid in enumerate(nm_imgs_train):
        image = load_img(dir_data + "/" + myid,
                         target_size=img_shape[:2])
        image = img_to_array(image)/255.0
        X_train.append(image)
    X_train = np.array(X_train)
    return(X_train)

#X_train = get_npdata(nm_imgs_train)
#print("X_train.shape = {}".format(X_train.shape))

#X_test  = get_npdata(nm_imgs_test)
#print("X_test.shape = {}".format(X_test.shape))

In [0]:
X_train = np.load('/content/drive/My Drive/train_data_facegen.npy')

In [0]:
def gen():
    model = Sequential()

    model.add(Dense(4*4*256,activation="relu",input_dim=100))
    model.add(Reshape((4,4,256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    # Output resolution, additional upsampling
    model.add(UpSampling2D())
    model.add(Conv2D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    #unquote pour images 64x64 
    model.add(UpSampling2D(size=(2,2)))
    model.add(Conv2D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    # Final CNN layer - out : 32x32
    model.add(Conv2D(3,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    return model

In [0]:
def disc():
    depth = 64
    dropout = 0.4
    img_rows = 64
    img_cols = 64
    img_channels = 3
    input_shape = (img_rows, img_cols, img_channels)
    
    model = Sequential()
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth=64
    model.add(Conv2D(depth, 5, strides=2, input_shape=input_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Conv2D(depth*2, 5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Conv2D(depth*4, 5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Conv2D(depth*8, 5, strides=1, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    # Out: probabilite
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

In [0]:
discriminant = disc()
discriminant.compile(loss='binary_crossentropy', 
                      optimizer=Adam(0.00146, 0.5), 
                      metrics=['accuracy'])
generateur = gen()
latent_dim = 100

In [0]:
def modele_competitif():
    model = Sequential()
    model.add(generateur)
    discriminant.trainable = False
    model.add(discriminant)
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(0.00146, 0.5), 
                  metrics=['accuracy'])
    discriminant.trainable = True
    return model

In [0]:
modele_comp = modele_competitif()

In [0]:
def save_images(cnt,noise):
  PREVIEW_ROWS = 4
  PREVIEW_COLS = 7
  PREVIEW_MARGIN = 16
  DATA_PATH = '/content'
  GENERATE_SQUARE = 64

  image_array = np.full(( 
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
      PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3), 
      255, dtype=np.uint8)
  
  generated_images = generateur.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
      for col in range(PREVIEW_COLS):
        r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join(DATA_PATH,'output')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"train-{cnt}.png")
  im = Image.fromarray(image_array)
  im.save(filename)

In [0]:
def train(train_epochs=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[32, latent_dim])
        for epoch in range(train_epochs):
            
            # sélection d'une moitié aléatoire d'images
            images_train = X_train[np.random.randint(0, X_train.shape[0], size=batch_size), :, :, :]
            
            # échantillonner du bruit aléatoire et générer des images
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_dim])
            images_fake = generateur.predict(noise)
            
            # entraîner le discriminant (1 = vraie, 0 = fausse)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = discriminant.train_on_batch(x, y)

            # entraîner le générateur
            y = np.ones([batch_size, 1])
            a_loss = modele_comp.train_on_batch(noise, y)
            
            log_msg = "%d: [D loss: %f, acc: %f]" % (epoch, d_loss[0], d_loss[1])
            log_msg = "%s  [A loss: %f, acc: %f]" % (log_msg, a_loss[0], a_loss[1])
            print(log_msg)
            if save_interval>0:
                if (epoch+1)%save_interval==0:
                    save_images(epoch,noise_input)

In [39]:
train(train_epochs=2000, batch_size=128, save_interval=100) 

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0: [D loss: 0.655620, acc: 0.718750]  [A loss: 0.784862, acc: 0.031250]
1: [D loss: 0.654235, acc: 0.726562]  [A loss: 0.781182, acc: 0.078125]
2: [D loss: 0.642726, acc: 0.769531]  [A loss: 0.752262, acc: 0.203125]
3: [D loss: 0.634730, acc: 0.718750]  [A loss: 0.762895, acc: 0.320312]
4: [D loss: 0.619973, acc: 0.699219]  [A loss: 0.729185, acc: 0.585938]
5: [D loss: 0.612104, acc: 0.691406]  [A loss: 0.707614, acc: 0.593750]
6: [D loss: 0.618535, acc: 0.558594]  [A loss: 0.627205, acc: 0.734375]
7: [D loss: 0.615091, acc: 0.566406]  [A loss: 0.548320, acc: 0.875000]
8: [D loss: 0.639632, acc: 0.531250]  [A loss: 0.518535, acc: 0.906250]
9: [D loss: 0.649416, acc: 0.531250]  [A loss: 0.572168, acc: 0.859375]
10: [D loss: 0.622937, acc: 0.542969]  [A loss: 0.656789, acc: 0.734375]
11: [D loss: 0.608807, acc: 0.605469]  [A loss: 0.703684, acc: 0.429688]
12: [D loss: 0.579676, acc: 0.773438]  [A loss: 0.701815, acc: 0.445312]
13: [D loss: 0.570493, acc: 0.824219]  [A loss: 0.692601, acc